# Наука про дані в хмарі: підхід "Azure ML SDK"

## Вступ

У цьому ноутбуці ми навчимося використовувати Azure ML SDK для тренування, розгортання та використання моделі через Azure ML.

Попередні вимоги:
1. Ви створили робоче середовище Azure ML.
2. Ви завантажили [набір даних про серцеву недостатність](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) в Azure ML.
3. Ви завантажили цей ноутбук у Azure ML Studio.

Наступні кроки:

1. Створити експеримент у наявному робочому середовищі.
2. Створити обчислювальний кластер.
3. Завантажити набір даних.
4. Налаштувати AutoML за допомогою AutoMLConfig.
5. Запустити експеримент AutoML.
6. Дослідити результати та отримати найкращу модель.
7. Зареєструвати найкращу модель.
8. Розгорнути найкращу модель.
9. Використати кінцеву точку.

## Імпорти, специфічні для Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Ініціалізація робочого простору
Ініціалізуйте об'єкт робочого простору з збереженої конфігурації. Переконайтеся, що файл конфігурації знаходиться за адресою .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Створення експерименту Azure ML

Давайте створимо експеримент під назвою 'aml-experiment' у робочій області, яку ми щойно ініціалізували.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Створення обчислювального кластеру
Вам потрібно створити [обчислювальну ціль](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) для запуску AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Дані
Переконайтеся, що ви завантажили набір даних до Azure ML і що ключ має таку ж назву, як і набір даних.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Конфігурація AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Запуск AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Розгортання найкращої моделі

Запустіть наступний код, щоб розгорнути найкращу модель. Ви можете переглянути стан розгортання в порталі Azure ML. Цей процес може зайняти кілька хвилин.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Використання кінцевої точки
Ви можете додати вхідні дані до наступного прикладу введення.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критичної інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
